# Análise Avançada - Dados Limpos (SILVER)
## Insights sobre Sinistros de Trânsito

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
print("✅ Bibliotecas carregadas")

In [ ]:
df = pd.read_csv('data/datatran2025.csv')
print(f"Dataset carregado: {len(df):,} registros")

In [ ]:
print("=" * 70)
print("PANORAMA GERAL")
print("=" * 70)

print(f"Acidentes: {len(df):,}")
print(f"Pessoas envolvidas: {df['pessoas'].sum():,}")
print(f"Óbitos: {df['mortos'].sum():,}")
print(f"Feridos: {df['feridos'].sum():,}")
print(f"Letalidade: {(df['mortos'].sum() / df['pessoas'].sum() * 100):.2f}%")

In [ ]:
acidentes_uf = df.groupby('uf').agg({'id': 'count', 'mortos': 'sum'}).sort_values('id', ascending=False).head(10)

plt.figure(figsize=(12, 6))
plt.barh(acidentes_uf.index, acidentes_uf['id'], color='#E63946')
plt.xlabel('Acidentes')
plt.title('Top 10 Estados', fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
clima = df.groupby('condicao_metereologica')['mortos'].sum().sort_values(ascending=False).head(8)

plt.figure(figsize=(12, 6))
plt.bar(range(len(clima)), clima.values, color='#457B9D')
plt.xticks(range(len(clima)), clima.index, rotation=45, ha='right')
plt.ylabel('Óbitos')
plt.title('Óbitos por Condição Climática', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
print("=" * 70)
print("PRINCIPAIS INSIGHTS")
print("=" * 70)

print(f"Estado mais crítico: {df['uf'].value_counts().index[0]}")
print(f"Dia mais perigoso: {df['dia_semana'].value_counts().index[0]}")
print(f"Tipo mais comum: {df['tipo_acidente'].value_counts().index[0]}")
print(f"Principal causa: {df['causa_acidente'].value_counts().index[0]}")

# Análise Avançada - Dados Normalizados (SILVER)
## Insights sobre Sinistros de Trânsito - Modelo Dimensional

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
print("✅ Bibliotecas carregadas")

In [ ]:
dim_local = pd.read_csv('data/dim_local.csv')
dim_tempo = pd.read_csv('data/dim_tempo.csv')
dim_condicoes = pd.read_csv('data/dim_condicoes.csv')
dim_tipo_acidente = pd.read_csv('data/dim_tipo_acidente.csv')
fato = pd.read_csv('data/fato_acidentes.csv')

print(f"Tabelas carregadas:")
print(f"  • DIM_LOCAL: {len(dim_local):,} registros")
print(f"  • DIM_TEMPO: {len(dim_tempo):,} registros")
print(f"  • DIM_CONDICOES: {len(dim_condicoes):,} registros")
print(f"  • DIM_TIPO_ACIDENTE: {len(dim_tipo_acidente):,} registros")
print(f"  • FATO_ACIDENTES: {len(fato):,} registros")

In [ ]:
df_completo = fato.merge(dim_local, on='id_local', how='left')
df_completo = df_completo.merge(dim_tempo, on='id_tempo', how='left')
df_completo = df_completo.merge(dim_condicoes, on='id_condicao', how='left')
df_completo = df_completo.merge(dim_tipo_acidente, on='id_tipo_acidente', how='left')

print(f"Dataset analítico construído: {len(df_completo):,} registros")
print(f"Colunas disponíveis: {len(df_completo.columns)}")

In [ ]:
print("=" * 70)
print("PANORAMA GERAL DOS ACIDENTES")
print("=" * 70)

total_acidentes = len(df_completo)
total_pessoas = df_completo['pessoas'].sum()
total_mortos = df_completo['mortos'].sum()
total_feridos = df_completo['feridos'].sum()
total_ilesos = df_completo['ilesos'].sum()
letalidade = (total_mortos / total_pessoas * 100) if total_pessoas > 0 else 0

print(f"\nAcidentes registrados: {total_acidentes:,}")
print(f"Pessoas envolvidas: {total_pessoas:,}")
print(f"Óbitos: {total_mortos:,}")
print(f"Feridos: {total_feridos:,}")
print(f"Ilesos: {total_ilesos:,}")
print(f"\nÍndice de letalidade: {letalidade:.2f}%")
print(f"Média de pessoas por acidente: {total_pessoas / total_acidentes:.2f}")

In [ ]:
acidentes_por_uf = df_completo.groupby('uf').agg({
    'id': 'count',
    'mortos': 'sum',
    'feridos': 'sum',
    'pessoas': 'sum'
}).rename(columns={'id': 'acidentes'}).sort_values('acidentes', ascending=False)

acidentes_por_uf['letalidade'] = (acidentes_por_uf['mortos'] / acidentes_por_uf['pessoas'] * 100).round(2)

print("=" * 70)
print("TOP 10 ESTADOS COM MAIS ACIDENTES")
print("=" * 70)
print(acidentes_por_uf.head(10).to_string())

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

top_10_uf = acidentes_por_uf.head(10)
axes[0, 0].barh(top_10_uf.index, top_10_uf['acidentes'], color='#E63946')
axes[0, 0].set_xlabel('Número de Acidentes', fontsize=11)
axes[0, 0].set_title('Estados com Mais Acidentes', fontsize=13, fontweight='bold')
axes[0, 0].invert_yaxis()

top_10_mortos = acidentes_por_uf.nlargest(10, 'mortos')
axes[0, 1].barh(top_10_mortos.index, top_10_mortos['mortos'], color='#457B9D')
axes[0, 1].set_xlabel('Número de Óbitos', fontsize=11)
axes[0, 1].set_title('Estados com Mais Óbitos', fontsize=13, fontweight='bold')
axes[0, 1].invert_yaxis()

acidentes_dia = df_completo['dia_semana'].value_counts()
ordem_dias = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']
acidentes_dia = acidentes_dia.reindex([d for d in ordem_dias if d in acidentes_dia.index])
axes[1, 0].bar(range(len(acidentes_dia)), acidentes_dia.values, color='#2A9D8F')
axes[1, 0].set_xticks(range(len(acidentes_dia)))
axes[1, 0].set_xticklabels([d[:3].upper() for d in acidentes_dia.index], rotation=0)
axes[1, 0].set_ylabel('Número de Acidentes', fontsize=11)
axes[1, 0].set_title('Distribuição por Dia da Semana', fontsize=13, fontweight='bold')

fase_dia = df_completo['fase_dia'].value_counts().head(5)
cores = ['#F4A261', '#E76F51', '#264653', '#2A9D8F', '#E9C46A']
axes[1, 1].pie(fase_dia.values, labels=fase_dia.index, autopct='%1.1f%%', 
               colors=cores, startangle=90)
axes[1, 1].set_title('Distribuição por Fase do Dia', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
clima_mortos = df_completo.groupby('condicao_metereologica').agg({
    'mortos': 'sum',
    'id': 'count'
}).rename(columns={'id': 'acidentes'}).sort_values('mortos', ascending=False).head(8)

print("=" * 70)
print("IMPACTO DAS CONDIÇÕES CLIMÁTICAS")
print("=" * 70)
print(clima_mortos.to_string())

fig, ax = plt.subplots(figsize=(14, 6))
x = range(len(clima_mortos))
ax.bar(x, clima_mortos['mortos'], color='#1D3557', alpha=0.8, label='Óbitos')
ax2 = ax.twinx()
ax2.plot(x, clima_mortos['acidentes'], color='#E63946', marker='o', linewidth=2, markersize=8, label='Acidentes')
ax.set_xticks(x)
ax.set_xticklabels(clima_mortos.index, rotation=45, ha='right')
ax.set_xlabel('Condição Meteorológica', fontsize=12)
ax.set_ylabel('Número de Óbitos', fontsize=12, color='#1D3557')
ax2.set_ylabel('Número de Acidentes', fontsize=12, color='#E63946')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.title('Óbitos e Acidentes por Condição Meteorológica', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
tipos = df_completo.groupby('tipo_acidente').agg({
    'id': 'count',
    'mortos': 'sum',
    'feridos': 'sum'
}).rename(columns={'id': 'acidentes'}).sort_values('acidentes', ascending=False).head(10)

print("=" * 70)
print("TIPOS DE ACIDENTES MAIS FREQUENTES")
print("=" * 70)
print(tipos.to_string())

fig, ax = plt.subplots(figsize=(14, 7))
tipos_top = tipos.head(8)
x = range(len(tipos_top))
width = 0.35
ax.bar([i - width/2 for i in x], tipos_top['mortos'], width, label='Óbitos', color='#C1121F')
ax.bar([i + width/2 for i in x], tipos_top['feridos'], width, label='Feridos', color='#FCA311')
ax.set_xticks(x)
ax.set_xticklabels(tipos_top.index, rotation=45, ha='right')
ax.set_ylabel('Número de Vítimas', fontsize=12)
ax.set_title('Vítimas por Tipo de Acidente', fontsize=14, fontweight='bold', pad=20)
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
causas = df_completo['causa_acidente'].value_counts().head(12)

print("=" * 70)
print("PRINCIPAIS CAUSAS DE ACIDENTES")
print("=" * 70)
for causa, qtd in causas.items():
    print(f"{causa}: {qtd:,}")

plt.figure(figsize=(14, 8))
y_pos = range(len(causas))
plt.barh(y_pos, causas.values, color='#006D77')
plt.yticks(y_pos, [c[:50] + '...' if len(c) > 50 else c for c in causas.index], fontsize=10)
plt.xlabel('Número de Acidentes', fontsize=12)
plt.title('Principais Causas de Acidentes de Trânsito', fontsize=14, fontweight='bold', pad=20)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("=" * 70)
print("SÍNTESE ANALÍTICA")
print("=" * 70)

uf_mais_acidentes = acidentes_por_uf.index[0]
uf_mais_mortes = acidentes_por_uf.nlargest(1, 'mortos').index[0]
dia_mais_perigoso = df_completo['dia_semana'].value_counts().index[0]
tipo_mais_comum = df_completo['tipo_acidente'].value_counts().index[0]
causa_principal = df_completo['causa_acidente'].value_counts().index[0]

print(f"\nEstado com mais acidentes: {uf_mais_acidentes}")
print(f"Estado com mais óbitos: {uf_mais_mortes}")
print(f"Dia da semana mais crítico: {dia_mais_perigoso}")
print(f"Tipo de acidente mais frequente: {tipo_mais_comum}")
print(f"Principal causa identificada: {causa_principal}")

gravidade_media = (total_mortos + total_feridos) / total_acidentes
print(f"\nGravidade média: {gravidade_media:.2f} vítimas/acidente")
print(f"Proporção óbitos/feridos: 1:{(total_feridos/total_mortos):.1f}")